In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# get category and subcategory list

In [39]:
base = requests.get('https://www.pinoyexchange.com/categories').text

In [40]:
bs = BeautifulSoup(base, 'lxml')

In [41]:
categories = []
for cat in bs.select('td.CategoryName')[1:]:
    category = cat.select('a')[-1].text
    href = cat.select('a')[-1]['href']
    category_url = requests.get(href).text
    category_bs = BeautifulSoup(category_url, 'lxml')
    for cat2 in category_bs.select('td.CategoryName')[1:]:
        category2 = cat2.select('a')[-1].text
        href2 = cat2.select('a')[-1]['href']
        categories.append({'category': category, 'subcategory': category2, 'href': href2})

In [47]:
cat_df = pd.DataFrame(categories)

In [45]:
cat_df.reset_index().to_csv('data/category_urls.csv', index=False)

# get posts given subcategory

In [91]:
def get_post_df(url, index):
    counter = 1
    running = pd.DataFrame()
    while True:
        print(counter)
        curr_url_text = url + '/p{}'.format(str(counter))
        sel = requests.get(curr_url_text).text
        bs_sel = BeautifulSoup(sel, 'lxml')
        to_append = pd.DataFrame(list(
            zip(
                [x['href'] for x in bs_sel.select('div.ItemContent.Discussion a')[::2]], # href
                [x.text for x in bs_sel.select('div.ItemContent.Discussion a')[::2]], # post
                [x.text for x in bs_sel.select('div.ItemContent.Discussion span.ViewCount span.Number')], # views
                [x.text for x in bs_sel.select('div.ItemContent.Discussion span.CommentCount span.Number')], # comments
                [x.text for x in bs_sel.select('div.ItemContent.Discussion span.DiscussionScore span.Number')], #discussion_score
                [x['datetime'] for x in bs_sel.select('div.ItemContent.Discussion time')] # date_last_post
            )
        ), columns = ['href', 'post', 'views', 'comments', 'discussion_score', 'date_last_post'])
        if to_append.shape[0] == 0:
            break
        running = pd.concat([running, to_append], axis= 0)
        counter += 1
    running['subcategory'] = index # identifier for subcategory
    running = running.reset_index() # index is identifier for post
    running.to_csv('data/subcategory_{}.csv'.format(index), index=False)

# get comments given post

In [47]:
post_df = pd.read_csv('data/subcategory_0.csv')

In [85]:
def get_comment_df(url, subcategory, index):
    counter = 1
    running = pd.DataFrame()
    while True:
        print(counter)
        curr_url_text = url + '/p{}'.format(str(counter))
        sel = requests.get(curr_url_text).text
        bs_sel = BeautifulSoup(sel, 'lxml')
        if counter == 1:
            to_append = pd.DataFrame(list(
                zip(
                [bs_sel.select('div.Meta.DiscussionMeta span.MItem.DateCreated time')[0]['datetime']] + [x['datetime'] for x in bs_sel.select('div.Item-Header.CommentHeader time')],
                [x.text for x in bs_sel.select('div.Message.userContent')]
                )
            ))
        else:
            to_append = pd.DataFrame(list(
                zip(
                [x['datetime'] for x in bs_sel.select('div.Item-Header.CommentHeader time')],
                [x.text for x in bs_sel.select('div.Message.userContent')]
                )
            ))
        if to_append.shape[0] == 0:
            break
        running = pd.concat([running, to_append], axis= 0)
        counter += 1
    running.columns = ['date', 'comment']
    running['subcategory'] = subcategory
    running['post_index'] = index
    running.to_csv('data/comments_{}_{}.csv'.format(subcategory, index), index=False)

# run the scraper

In [92]:
cat_df = pd.read_csv('data/category_urls.csv')

In [ ]:
# saves posts per subcategory in a csv file
subcategories_to_go_over = cat_df[['href', 'index', 'category', 'subcategory']].values
for x in subcategories_to_go_over:
    print('Category: {}, Subcategory: {}'.format(x[2], x[3]))
    get_post_df(x[0], x[1])

In [ ]:
# saves comments per (subcategory and post id) in a csv file
for x in range(92):
    subcat_df = pd.read_csv('data/subcategory_{}.csv'.format(index))
    posts_to_go_over = subcat_df[['href', 'subcategory', 'index', 'post']]
    for y in posts_to_go_over:
        print('Subcategory ID: {}, Post: {}'.format(y[1], y[3]))
        get_comment_df(y[0], y[1], y[2])